In [1]:
import nltk 
# nltk.download('gutenberg')
# from nltk.corpus import gutenberg
# import pandas as pd
# data=gutenberg.raw('shakespeare-hamlet.txt')
# with open('hamlet.txt','w') as file:
#     file.write(data)

In [2]:
#  data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import  pad_sequences
from sklearn.model_selection import  train_test_split

# load the data set
with open ('hamlet.txt','r') as file:
    text=file.read().lower()

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
# tokenizer.word_index
total_words


4818

In [3]:
#  spliting the words 
inputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        inputsequences.append(n_gram_sequence)
        

## What the code generates:
corpus =I love predict Nlp

[12, 45] → Means: "Given I, predict love"

[12, 45, 78] → Means: "Given I love, predict NLP"

In [4]:
inputsequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [5]:
max_sequence_len=max([len(x) for x in inputsequences])
max_sequence_len

14

In [6]:
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_sequence_len,padding='pre'))
inputsequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [7]:
import tensorflow as tf
x,y=inputsequences[:,:-1],inputsequences[:,-1]

In [8]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [9]:
y

array([ 687,    4,   45, ..., 1047,    4,  193])

In [10]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y
#  we converted the output same as input so we split with number of cati,same like one hot encodeing u can visuilize

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
# print(type(x_train), x_train.shape)
# print(type(y_train), y_train.shape)
# print(type(x_test), x_test.shape)
# print(type(y_test), y_test.shape)


In [12]:
# print(np.isnan(x_train).any(), np.isnan(y_train).any())
# print(np.isnan(x_test).any(), np.isnan(y_test).any())


In [13]:
print(x.shape, y.shape)


(25732, 13) (25732, 4818)


In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

## training


In [15]:
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import  Embedding,LSTM,Dense,Dropout

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

model.build(input_shape=(None, max_sequence_len))
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# x_trian
history=model.fit(x_train,y_train,epochs=70,validation_data=(x_test,y_test),verbose=1)

Epoch 1/70


644/644 [==============================] - 48s 64ms/step - loss: 6.9142 - accuracy: 0.0320 - val_loss: 6.7160 - val_accuracy: 0.0394
Epoch 2/70
644/644 [==============================] - 41s 64ms/step - loss: 6.4750 - accuracy: 0.0365 - val_loss: 6.7883 - val_accuracy: 0.0476
Epoch 3/70
644/644 [==============================] - 47s 72ms/step - loss: 6.3412 - accuracy: 0.0428 - val_loss: 6.8106 - val_accuracy: 0.0550
Epoch 4/70
644/644 [==============================] - 44s 69ms/step - loss: 6.2049 - accuracy: 0.0483 - val_loss: 6.8587 - val_accuracy: 0.0552
Epoch 5/70
644/644 [==============================] - 42s 66ms/step - loss: 6.0689 - accuracy: 0.0507 - val_loss: 6.8746 - val_accuracy: 0.0542
Epoch 6/70
644/644 [==============================] - 43s 67ms/step - loss: 5.9223 - accuracy: 0.0615 - val_loss: 6.9012 - val_accuracy: 0.0659
Epoch 7/70
644/644 [==============================] - 45s 70ms/step - loss: 5.7810 - accuracy: 0.0708 - val_loss: 6.9364 - val_accurac

In [17]:
# x_trian
history=model.fit(x_train,y_train,initial_epoch=70,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 71/100
644/644 [==============================] - 39s 60ms/step - loss: 2.0459 - accuracy: 0.5399 - val_loss: 12.7998 - val_accuracy: 0.0521
Epoch 72/100
644/644 [==============================] - 39s 61ms/step - loss: 2.0257 - accuracy: 0.5430 - val_loss: 12.8852 - val_accuracy: 0.0517
Epoch 73/100
644/644 [==============================] - 37s 58ms/step - loss: 2.0017 - accuracy: 0.5463 - val_loss: 12.9171 - val_accuracy: 0.0511
Epoch 74/100
644/644 [==============================] - 38s 60ms/step - loss: 1.9809 - accuracy: 0.5519 - val_loss: 12.9701 - val_accuracy: 0.0530
Epoch 75/100
644/644 [==============================] - 39s 61ms/step - loss: 1.9564 - accuracy: 0.5567 - val_loss: 13.0286 - val_accuracy: 0.0558
Epoch 76/100
644/644 [==============================] - 38s 60ms/step - loss: 1.9387 - accuracy: 0.5595 - val_loss: 13.0663 - val_accuracy: 0.0507
Epoch 77/100
644/644 [==============================] - 39s 60ms/step - loss: 1.9178 - accuracy: 0.5612 - val_loss: 13

In [18]:
# x_trian
history=model.fit(x_train,y_train,initial_epoch=100,epochs=125,validation_data=(x_test,y_test),verbose=1)

Epoch 101/125
644/644 [==============================] - 39s 60ms/step - loss: 1.5171 - accuracy: 0.6529 - val_loss: 14.2515 - val_accuracy: 0.0490
Epoch 102/125
644/644 [==============================] - 37s 57ms/step - loss: 1.5102 - accuracy: 0.6530 - val_loss: 14.2873 - val_accuracy: 0.0505
Epoch 103/125
644/644 [==============================] - 41s 63ms/step - loss: 1.4922 - accuracy: 0.6551 - val_loss: 14.3487 - val_accuracy: 0.0503
Epoch 104/125
644/644 [==============================] - 39s 60ms/step - loss: 1.4793 - accuracy: 0.6587 - val_loss: 14.3686 - val_accuracy: 0.0519
Epoch 105/125
644/644 [==============================] - 37s 58ms/step - loss: 1.4654 - accuracy: 0.6612 - val_loss: 14.4209 - val_accuracy: 0.0474
Epoch 106/125
644/644 [==============================] - 40s 62ms/step - loss: 1.4535 - accuracy: 0.6639 - val_loss: 14.4755 - val_accuracy: 0.0534
Epoch 107/125
644/644 [==============================] - 38s 60ms/step - loss: 1.4428 - accuracy: 0.6710 - val_l

In [19]:
# x_trian
history=model.fit(x_train,y_train,initial_epoch=125,epochs=150,validation_data=(x_test,y_test),verbose=1)

Epoch 126/150
644/644 [==============================] - 39s 60ms/step - loss: 1.2407 - accuracy: 0.7113 - val_loss: 15.2902 - val_accuracy: 0.0488
Epoch 127/150
644/644 [==============================] - 37s 58ms/step - loss: 1.2302 - accuracy: 0.7140 - val_loss: 15.3244 - val_accuracy: 0.0462
Epoch 128/150
644/644 [==============================] - 37s 58ms/step - loss: 1.2205 - accuracy: 0.7151 - val_loss: 15.3539 - val_accuracy: 0.0470
Epoch 129/150
644/644 [==============================] - 39s 61ms/step - loss: 1.2180 - accuracy: 0.7148 - val_loss: 15.3191 - val_accuracy: 0.0495
Epoch 130/150
644/644 [==============================] - 38s 58ms/step - loss: 1.2096 - accuracy: 0.7164 - val_loss: 15.4208 - val_accuracy: 0.0484
Epoch 131/150
644/644 [==============================] - 38s 59ms/step - loss: 1.1998 - accuracy: 0.7201 - val_loss: 15.4022 - val_accuracy: 0.0476
Epoch 132/150
644/644 [==============================] - 40s 63ms/step - loss: 1.1880 - accuracy: 0.7226 - val_l

In [20]:
# x_trian
history=model.fit(x_train,y_train,initial_epoch=150,epochs=180,validation_data=(x_test,y_test),verbose=1)

Epoch 151/180
644/644 [==============================] - 35s 54ms/step - loss: 1.0575 - accuracy: 0.7490 - val_loss: 16.0918 - val_accuracy: 0.0466
Epoch 152/180
644/644 [==============================] - 35s 54ms/step - loss: 1.0489 - accuracy: 0.7497 - val_loss: 16.1322 - val_accuracy: 0.0492
Epoch 153/180
644/644 [==============================] - 37s 57ms/step - loss: 1.0477 - accuracy: 0.7515 - val_loss: 16.1161 - val_accuracy: 0.0451
Epoch 154/180
644/644 [==============================] - 36s 56ms/step - loss: 1.0424 - accuracy: 0.7510 - val_loss: 16.1549 - val_accuracy: 0.0495
Epoch 155/180
644/644 [==============================] - 37s 58ms/step - loss: 1.0342 - accuracy: 0.7540 - val_loss: 16.1762 - val_accuracy: 0.0460
Epoch 156/180
644/644 [==============================] - 35s 54ms/step - loss: 1.0255 - accuracy: 0.7567 - val_loss: 16.2384 - val_accuracy: 0.0499
Epoch 157/180
644/644 [==============================] - 37s 57ms/step - loss: 1.0288 - accuracy: 0.7560 - val_l

In [21]:
# x_trian
history=model.fit(x_train,y_train,initial_epoch=180,epochs=200,validation_data=(x_test,y_test),verbose=1)

Epoch 181/200
644/644 [==============================] - 18s 29ms/step - loss: 0.9184 - accuracy: 0.7781 - val_loss: 16.8852 - val_accuracy: 0.0455
Epoch 182/200
644/644 [==============================] - 19s 30ms/step - loss: 0.9234 - accuracy: 0.7756 - val_loss: 16.8701 - val_accuracy: 0.0443
Epoch 183/200
644/644 [==============================] - 20s 31ms/step - loss: 0.9096 - accuracy: 0.7795 - val_loss: 16.9249 - val_accuracy: 0.0464
Epoch 184/200
644/644 [==============================] - 20s 31ms/step - loss: 0.9048 - accuracy: 0.7793 - val_loss: 16.9761 - val_accuracy: 0.0445
Epoch 185/200
644/644 [==============================] - 20s 30ms/step - loss: 0.9030 - accuracy: 0.7805 - val_loss: 16.9746 - val_accuracy: 0.0488
Epoch 186/200
644/644 [==============================] - 19s 30ms/step - loss: 0.9003 - accuracy: 0.7822 - val_loss: 17.0543 - val_accuracy: 0.0464
Epoch 187/200
644/644 [==============================] - 20s 30ms/step - loss: 0.8978 - accuracy: 0.7819 - val_l

In [43]:
# x_trian
history=model.fit(x_train,y_train,initial_epoch=200,epochs=250,validation_data=(x_test,y_test),verbose=1)

Epoch 201/250
644/644 [==============================] - 17s 26ms/step - loss: 0.8568 - accuracy: 0.7866 - val_loss: 17.3215 - val_accuracy: 0.0486
Epoch 202/250
644/644 [==============================] - 16s 25ms/step - loss: 0.8614 - accuracy: 0.7880 - val_loss: 17.3789 - val_accuracy: 0.0474
Epoch 203/250
644/644 [==============================] - 17s 27ms/step - loss: 0.8506 - accuracy: 0.7920 - val_loss: 17.3569 - val_accuracy: 0.0460
Epoch 204/250
644/644 [==============================] - 18s 28ms/step - loss: 0.8497 - accuracy: 0.7924 - val_loss: 17.4445 - val_accuracy: 0.0464
Epoch 205/250
644/644 [==============================] - 18s 28ms/step - loss: 0.8394 - accuracy: 0.7926 - val_loss: 17.4790 - val_accuracy: 0.0429
Epoch 206/250
644/644 [==============================] - 19s 29ms/step - loss: 0.8435 - accuracy: 0.7918 - val_loss: 17.4790 - val_accuracy: 0.0447
Epoch 207/250
644/644 [==============================] - 19s 29ms/step - loss: 0.8342 - accuracy: 0.7922 - val_l

In [44]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):]
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [ ]:
input_text="Hello this is me as "
print(f"Input text : {input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word prediction : {next_word}")

Input text : Hello this is me as i 
Next word prediction : finde


In [45]:
model.save("next_word_lstm.h5")
import pickle
with open('tokenizer.pickle','wb') as handle :
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)